In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import csv
import time
from datetime import datetime, timedelta

# 1. Filtering out COVID posts

In [ ]:
df = pd.read_csv('last240Days_lang.csv', encoding='utf-8', parse_dates=[5,])
df.head()

,Post ID,Title,URL,Author,Score,Publish Date,Total number of comments,Permalink,Flair,Language
0,e0xx7y,US committed to securing trade deal with China...,https://www.businessinsider.com/us-warns-chine...,viva_la_vinyl,1,2019-11-24 13:27:28,3,/r/worldnews/comments/e0xx7y/us_committed_to_s...,NaN,b'en'
1,e0xyfw,Pope Francis orders world leaders to give up a...,https://www.france24.com/en/20191124-pope-fran...,viva_la_vinyl,1,2019-11-24 13:31:12,92,/r/worldnews/comments/e0xyfw/pope_francis_orde...,NaN,b'en'
2,e0xyox,‘Racist and Islamopobic': Man Brutally Attacks...,https://www.news18.com/news/buzz/racist-and-is...,420catnip,1,2019-11-24 13:31:48,0,/r/worldnews/comments/e0xyox/racist_and_islamo...,NaN,b'en'
3,e0xypu,Global 5G deal poses significant threat to wea...,https://www.washingtonpost.com/weather/2019/11...,Philo1927,1,2019-11-24 13:31:54,1,/r/worldnews/comments/e0xypu/global_5g_deal_po...,NaN,b'en'
4,e0xywy,"DisguisedToast, a top Twitch streamer, is movi...",https://smartrobotics.com.ng/2019/11/22/disgui...,jbawo,1,2019-11-24 13:32:27,0,/r/worldnews/comments/e0xywy/disguisedtoast_a_...,NaN,b'en'


In [ ]:
langs = [lang.lstrip("'b''").rstrip("'") for lang in df["Language"].values]
df["Language"] = langs
df.head()

,Post ID,Title,URL,Author,Score,Publish Date,Total number of comments,Permalink,Flair,Language
0,e0xx7y,US committed to securing trade deal with China...,https://www.businessinsider.com/us-warns-chine...,viva_la_vinyl,1,2019-11-24 13:27:28,3,/r/worldnews/comments/e0xx7y/us_committed_to_s...,NaN,en
1,e0xyfw,Pope Francis orders world leaders to give up a...,https://www.france24.com/en/20191124-pope-fran...,viva_la_vinyl,1,2019-11-24 13:31:12,92,/r/worldnews/comments/e0xyfw/pope_francis_orde...,NaN,en
2,e0xyox,‘Racist and Islamopobic': Man Brutally Attacks...,https://www.news18.com/news/buzz/racist-and-is...,420catnip,1,2019-11-24 13:31:48,0,/r/worldnews/comments/e0xyox/racist_and_islamo...,NaN,en
3,e0xypu,Global 5G deal poses significant threat to wea...,https://www.washingtonpost.com/weather/2019/11...,Philo1927,1,2019-11-24 13:31:54,1,/r/worldnews/comments/e0xypu/global_5g_deal_po...,NaN,en
4,e0xywy,"DisguisedToast, a top Twitch streamer, is movi...",https://smartrobotics.com.ng/2019/11/22/disgui...,jbawo,1,2019-11-24 13:32:27,0,/r/worldnews/comments/e0xywy/disguisedtoast_a_...,NaN,en


In [ ]:
df_eng = df[df["Language"] == 'en']
df_eng.head()

,Post ID,Title,URL,Author,Score,Publish Date,Total number of comments,Permalink,Flair,Language
0,e0xx7y,US committed to securing trade deal with China...,https://www.businessinsider.com/us-warns-chine...,viva_la_vinyl,1,2019-11-24 13:27:28,3,/r/worldnews/comments/e0xx7y/us_committed_to_s...,NaN,en
1,e0xyfw,Pope Francis orders world leaders to give up a...,https://www.france24.com/en/20191124-pope-fran...,viva_la_vinyl,1,2019-11-24 13:31:12,92,/r/worldnews/comments/e0xyfw/pope_francis_orde...,NaN,en
2,e0xyox,‘Racist and Islamopobic': Man Brutally Attacks...,https://www.news18.com/news/buzz/racist-and-is...,420catnip,1,2019-11-24 13:31:48,0,/r/worldnews/comments/e0xyox/racist_and_islamo...,NaN,en
3,e0xypu,Global 5G deal poses significant threat to wea...,https://www.washingtonpost.com/weather/2019/11...,Philo1927,1,2019-11-24 13:31:54,1,/r/worldnews/comments/e0xypu/global_5g_deal_po...,NaN,en
4,e0xywy,"DisguisedToast, a top Twitch streamer, is movi...",https://smartrobotics.com.ng/2019/11/22/disgui...,jbawo,1,2019-11-24 13:32:27,0,/r/worldnews/comments/e0xywy/disguisedtoast_a_...,NaN,en


In [ ]:
# COVID-19 tag was introduced in March, let's see how many non-tagged entries there are
import re
df_eng.loc[((df_eng['Title'].str.contains('virus|sars|covid|pandemic|epidemy', flags=re.I, regex=True)) & (df_eng['Flair'].isna()) | (df_eng["Flair"] == "COVID-19")), "isCovid"] = True

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
df_eng.loc[df_eng["isCovid"] == True].head(40)

,Post ID,Title,URL,Author,Score,Publish Date,Total number of comments,Permalink,Flair,Language,isCovid
1622,e1x28q,More polio cases now caused by vaccine than by...,https://apnews.com/7d8b0e32efd0480fbd12acf2772...,nootPuber,1,2019-11-26 13:05:49,7,/r/worldnews/comments/e1x28q/more_polio_cases_...,NaN,en,True
1731,e1yr53,WHO: More polio cases currently caused by vacc...,http://nationalpost.com/health/more-polio-case...,Mister_Kurtz,1,2019-11-26 15:36:14,5,/r/worldnews/comments/e1yr53/who_more_polio_ca...,NaN,en,True
2519,e2g5l9,More Polio cases now caused by vaccine than by...,https://abcnews.go.com/Health/wireStory/polio-...,TheHappyGospel,1,2019-11-27 14:41:12,15,/r/worldnews/comments/e2g5l9/more_polio_cases_...,NaN,en,True
2643,e2ie69,HIV-positive sperm bank launched in New Zealan...,https://www.dw.com/en/worlds-first-hiv-positiv...,Molire,1,2019-11-27 17:26:21,15,/r/worldnews/comments/e2ie69/hivpositive_sperm...,NaN,en,True
3218,e2w17g,A measles epidemic in Samoa has killed 39 peop...,https://www.theguardian.com/world/2019/nov/28/...,ManiaforBeatles,1,2019-11-28 11:00:13,3,/r/worldnews/comments/e2w17g/a_measles_epidemi...,NaN,en,True
3381,e2y2qd,Oh No! Health Workers Fighting Ebola Virus Kil...,http://fimtvblog.blogspot.com/2019/11/oh-no-he...,mimelord120,1,2019-11-28 14:33:37,1,/r/worldnews/comments/e2y2qd/oh_no_health_work...,NaN,en,True
5497,e4i0x8,"Between Black Friday and Cyber Monday, pope co...",https://www.reuters.com/article/us-usa-holiday...,pantangeli,1,2019-12-01 15:23:35,476,/r/worldnews/comments/e4i0x8/between_black_fri...,NaN,en,True
6003,e4w2l4,Samoa ordered a government shutdown to help co...,https://www.france24.com/en/20191202-governmen...,DoremusJessup,1,2019-12-02 09:18:47,7,/r/worldnews/comments/e4w2l4/samoa_ordered_a_g...,NaN,en,True
6831,e5dalu,Linda Sarsour: Israel is ‘built on’ idea that ...,https://www.washingtonexaminer.com/news/linda-...,blackstonebite,1,2019-12-03 08:02:51,261,/r/worldnews/comments/e5dalu/linda_sarsour_isr...,NaN,en,True
8241,e64yu9,Sarsour attempts to clarify remark that Israel...,https://worldisraelnews.com/sarsour-attempts-t...,Nemesis4bigotry,1,2019-12-04 21:06:36,3,/r/worldnews/comments/e64yu9/sarsour_attempts_...,NaN,en,True


In [ ]:
df_target = df_eng[(df_eng["isCovid"] == True) & (df_eng["Publish Date"] > np.datetime64('2019-12-31'))]
df_target.head(10)

,Post ID,Title,URL,Author,Score,Publish Date,Total number of comments,Permalink,Flair,Language,isCovid
27569,ei54wv,China investigates SARS-like virus as 27 struc...,https://www.taiwannews.com.tw/en/news/3847781,crabmeat2,1,2019-12-31 17:35:02,1,/r/worldnews/comments/ei54wv/china_investigate...,NaN,en,True
27773,eib9ut,A scary unidentified virus is spreading in China,https://bgr.com/2019/12/31/china-pneumonia-sic...,skippy_smooth,1,2020-01-01 01:21:23,70,/r/worldnews/comments/eib9ut/a_scary_unidentif...,NaN,en,True
28122,eikpfp,China probes mystery pneumonia outbreak amid S...,https://news.yahoo.com/china-probes-pneumonia-...,LefthandedLunatic,1,2020-01-01 18:14:34,5,/r/worldnews/comments/eikpfp/china_probes_myst...,NaN,en,True
28211,eimhbw,China investigates SARS-like virus as dozens s...,https://www.dw.com/en/china-investigates-sars-...,dont_come_any_closer,1,2020-01-01 20:29:24,7,/r/worldnews/comments/eimhbw/china_investigate...,NaN,en,True
28527,eiwl41,China on high alert following outbreak of SARS...,https://www.news.com.au/lifestyle/health/healt...,vik0_tal,1,2020-01-02 11:23:58,2,/r/worldnews/comments/eiwl41/china_on_high_ale...,NaN,en,True
28535,eiwodn,China investigates SARS-like virus as dozens s...,https://m.dw.com/en/china-investigates-sars-li...,vik0_tal,1,2020-01-02 11:35:38,0,/r/worldnews/comments/eiwodn/china_investigate...,NaN,en,True
28786,ej0uxq,Mystery pneumonia outbreak in China sparks fea...,https://www.cbsnews.com/news/china-sars-virus-...,iuseptt,1,2020-01-02 18:05:16,5,/r/worldnews/comments/ej0uxq/mystery_pneumonia...,NaN,en,True
29617,ejf9me,Asian governments on alert over mysterious vir...,https://asia.nikkei.com/Politics/Asian-governm...,cc_hk,1722,2020-01-03 14:15:33,413,/r/worldnews/comments/ejf9me/asian_governments...,NaN,en,True
29711,ejgkgr,China pneumonia outbreak: Mystery virus probed...,https://www.bbc.co.uk/news/world-asia-china-50...,dototoohard,30,2020-01-03 16:05:26,3,/r/worldnews/comments/ejgkgr/china_pneumonia_o...,NaN,en,True
29713,ejgkvp,China pneumonia outbreak: Mystery virus probed...,https://www.bbc.co.uk/news/world-asia-china-50...,Evil_Japanese_Pie,11,2020-01-03 16:06:14,1,/r/worldnews/comments/ejgkvp/china_pneumonia_o...,NaN,en,True


# 2. Emoji classification

In [ ]:
titles = df_target.Title.tolist()
titles[:5]

['China investigates SARS-like virus as 27 struck by pneumonia in Wuhan, Hubei Provence.',
 'A scary unidentified virus is spreading in China',
 'China probes mystery pneumonia outbreak amid SARS fears',
 'China investigates SARS-like virus as dozens struck by pneumonia',
 'China on high alert following outbreak of SARS-like virus']

In [ ]:
pip3 install torch==1.0.1 -f https://download.pytorch.org/whl/cpu/stable 
git clone https://github.com/huggingface/torchMoji
import os
os.chdir('torchMoji')
pip3 install -e .
#if you restart the package, the notebook risks to crash on a loop
#I did not restart and worked fine

Looking in links: https://download.pytorch.org/whl/cpu/stable
     |████████████████████████████████| 560.1MB 29kB/s 
ERROR: torchvision 0.6.1+cu101 has requirement torch==1.5.1, but you'll have torch 1.0.1 which is incompatible.
  Found existing installation: torch 1.5.1+cu101
    Uninstalling torch-1.5.1+cu101:
      Successfully uninstalled torch-1.5.1+cu101
Cloning into 'torchMoji'...
remote: Enumerating objects: 143, done.
remote: Total 143 (delta 0), reused 0 (delta 0), pack-reused 143
Receiving objects: 100% (143/143), 2.41 MiB | 15.06 MiB/s, done.
Resolving deltas: 100% (49/49), done.
Obtaining file:///content/torchMoji
     |████████████████████████████████| 17.0MB 1.3MB/s 
     |████████████████████████████████| 48.2MB 81kB/s 
     |████████████████████████████████| 12.4MB 44.0MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
  Created wheel for emoji: filename=emoji-0.4.5-cp36-none-any.whl size=38198 sha256=dce18e20e7389e888ed15c2defd441c0a1b7aa7eaf2bee2825c1377685

In [ ]:
python3 scripts/download_weights.py

About to download the pretrained weights file from https://www.dropbox.com/s/q8lax9ary32c7t9/pytorch_model.bin?dl=0#
The size of the file is roughly 85MB. Continue? [y/n]
y
Downloading...
Running system call: wget https://www.dropbox.com/s/q8lax9ary32c7t9/pytorch_model.bin?dl=0# -O /content/torchMoji/model/pytorch_model.bin
--2020-07-22 09:57:14--  https://www.dropbox.com/s/q8lax9ary32c7t9/pytorch_model.bin?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.1, 2620:100:601d:1::a27d:501
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/q8lax9ary32c7t9/pytorch_model.bin [following]
--2020-07-22 09:57:15--  https://www.dropbox.com/s/raw/q8lax9ary32c7t9/pytorch_model.bin
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf576de82020430262c0d1c6c50.dl.dropboxusercontent.com/cd/0/inline/A7-gQ1gnzqIUcMY1DL

In [ ]:
import numpy as np
import emoji, json
from torchmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH
from torchmoji.sentence_tokenizer import SentenceTokenizer
from torchmoji.model_def import torchmoji_emojis
  
EMOJIS = ":joy: :unamused: :weary: :sob: :heart_eyes: :pensive: :ok_hand: :blush: :heart: :smirk: :grin: :notes: :flushed: :100: :sleeping: :relieved: :relaxed: :raised_hands: :two_hearts: :expressionless: :sweat_smile: :pray: :confused: :kissing_heart: :heartbeat: :neutral_face: :information_desk_person: :disappointed: :see_no_evil: :tired_face: :v: :sunglasses: :rage: :thumbsup: :cry: :sleepy: :yum: :triumph: :hand: :mask: :clap: :eyes: :gun: :persevere: :smiling_imp: :sweat: :broken_heart: :yellow_heart: :musical_note: :speak_no_evil: :wink: :skull: :confounded: :smile: :stuck_out_tongue_winking_eye: :angry: :no_good: :muscle: :facepunch: :purple_heart: :sparkling_heart: :blue_heart: :grimacing: :sparkles:".split(' ')
model = torchmoji_emojis(PRETRAINED_PATH)
with open(VOCAB_PATH, 'r') as f:
  vocabulary = json.load(f)
st = SentenceTokenizer(vocabulary, 30)

def deepmojify(sentence,top_n =5):
  tokenized, _, _ = st.tokenize_sentences([sentence])
  prob = model(tokenized)[0]
  emoji_ids = top_elements(prob, top_n)
  emojis = map(lambda x: EMOJIS[x], emoji_ids)
  return emoji.emojize(f"{' '.join(emojis)}", use_aliases=True)

def top_elements(array, k):
  ind = np.argpartition(array, -k)[-k:]
  return ind[np.argsort(array[ind])][::-1]

/content/torchMoji/torchmoji/model_def.py:159: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.embed.weight.data, a=-0.5, b=0.5)
/content/torchMoji/torchmoji/model_def.py:161: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(t)
/content/torchMoji/torchmoji/model_def.py:163: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  nn.init.orthogonal(t)
/content/torchMoji/torchmoji/model_def.py:165: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(t, 0)
/content/torchMoji/torchmoji/model_def.py:167: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.output_layer[0].weight.data)


In [ ]:
emoticons = len(titles)*[""]
for index, _ in enumerate(titles):
  emoticons[index] = (deepmojify(_, top_n =5))
  if index % 5000 == 0:
    print(index)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000


In [ ]:
df_target["Emoji"] = emoticons
df_target.to_csv('last240Days_emo.csv', index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# 3. Processing emoji data

In [ ]:
df = pd.read_csv('last240Days_emo.csv', parse_dates=[5,])
df.head()

,Post ID,Title,URL,Author,Score,Publish Date,Total number of comments,Permalink,Flair,Language,isCovid,Day,Emoji
0,ei54wv,China investigates SARS-like virus as 27 struc...,https://www.taiwannews.com.tw/en/news/3847781,crabmeat2,1,2019-12-31 17:35:02,1,/r/worldnews/comments/ei54wv/china_investigate...,NaN,en,True,2019-12-31,😷 😖 😣 😠 😓
1,eib9ut,A scary unidentified virus is spreading in China,https://bgr.com/2019/12/31/china-pneumonia-sic...,skippy_smooth,1,2020-01-01 01:21:23,70,/r/worldnews/comments/eib9ut/a_scary_unidentif...,NaN,en,True,2020-01-01,😖 😷 😳 😣 😫
2,eikpfp,China probes mystery pneumonia outbreak amid S...,https://news.yahoo.com/china-probes-pneumonia-...,LefthandedLunatic,1,2020-01-01 18:14:34,5,/r/worldnews/comments/eikpfp/china_probes_myst...,NaN,en,True,2020-01-01,🎶 😷 🎵 😖 💀
3,eimhbw,China investigates SARS-like virus as dozens s...,https://www.dw.com/en/china-investigates-sars-...,dont_come_any_closer,1,2020-01-01 20:29:24,7,/r/worldnews/comments/eimhbw/china_investigate...,NaN,en,True,2020-01-01,😷 😖 😣 😪 😓
4,eiwl41,China on high alert following outbreak of SARS...,https://www.news.com.au/lifestyle/health/healt...,vik0_tal,1,2020-01-02 11:23:58,2,/r/worldnews/comments/eiwl41/china_on_high_ale...,NaN,en,True,2020-01-02,😷 😖 😳 😐 😡


In [ ]:
emojis = df.Emoji.tolist()
days = df.Day.tolist()
len(df)

54936

In [ ]:
emojis[0].split(" ")

['😷', '😖', '😣', '😠', '😓']

In [ ]:
top_levels = [5, 3, 2, 1]
df_list = []
for level in top_levels:
  emo_dict = {}
  for top_n in emojis:
    for emo in top_n.split(" ")[:level+1]:
      if emo not in emo_dict:
        emo_dict[emo] = 0
      emo_dict[emo] += 1
  df_list.append(pd.DataFrame.from_dict(emo_dict, orient='index', columns=[f"Count_top{level}"]))

In [ ]:
df_emo = df_list[0]
for df in df_list[1:]:
  df_emo = pd.concat([df_emo, df], axis=1)
df_emo = df_emo.sort_values(by='Count_top1', ascending=False)
df_emo

,Count_top5,Count_top3,Count_top2,Count_top1
👍,29625,27034,23554,18786
😷,13821,12934,11904,10409
😡,15451,13916,12075,9713
💪,12777,11197,9268,6637
😢,11185,9524,7697,5737
...,...,...,...,...
😫,174,77,34,14
😏,133,82,47,14
😔,469,248,57,6
💕,31,17,9,3


In [ ]:
df_emo.to_csv('/content/gdrive/My Drive/Colab Notebooks/KoronaScience/data/emoji_covid_full_summary.csv')